In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import glob
import numpy as np
import h5py

sequence_len = 64
embed_dim  = 128
full_len = sequence_len*embed_dim
data_cut = 10005 - full_len
ff_dim = 1024
num_heads = 6
File_Path = "Test_data.h5"
checkpoint_path = "T-encoder_EG.ckpt"

data= h5py.File('Test_data.h5', 'r')
stack=data['data'][:,:]

data = stack[:,:-data_cut]/100
y_data = stack[:,-4]-1
y_data = tf.keras.utils.to_categorical(y_data, num_classes = 101)

x_data = data.reshape(-1,sequence_len,embed_dim)

In [2]:
class PositionEmbedding(layers.Layer):
    def __init__(self, sequence_len, embed_dim):
        super(PositionEmbedding, self).__init__()
        self.pos_emb = layers.Embedding(input_dim=sequence_len, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=sequence_len, delta=1)
        positions = self.pos_emb(positions)
        return x + positions

In [3]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [4]:
inputs = tf.keras.Input(shape=[sequence_len, embed_dim])

embedding_layer = PositionEmbedding(sequence_len, embed_dim)
x = embedding_layer(inputs)

transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(inputs)

transformer_block_2 = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block_2(x)

x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(1024, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(101, activation="softmax")(x)

T_encoder = keras.Model(inputs=inputs, outputs=outputs)
opt = tf.keras.optimizers.Adam(learning_rate=0.0002)
T_encoder.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(5)])
T_encoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 128)]         0         
_________________________________________________________________
transformer_block (Transform (None, 64, 128)           659456    
_________________________________________________________________
transformer_block_1 (Transfo (None, 64, 128)           659456    
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              132096    
_________________________________________________________________
dropout_5 (Dropout)          (None, 1024)              0     

In [5]:
T_encoder.load_weights(checkpoint_path)
T_encoder.evaluate(x_data, y_data)

4/4 [==============================] - 1s 4ms/step - loss: 0.9463 - accuracy: 0.7300 - top_k_categorical_accuracy: 0.9600


[0.9463157057762146, 0.7300000190734863, 0.9599999785423279]